In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, TimeSeriesSplit
# Calculating forecast accuracy
from sklearn.metrics import mean_absolute_error, mean_squared_error
import optuna

In [10]:
# Load your data
# Assuming you have a CSV file with columns: 'date', 'dengue_patient_count', 'precipitation', 'wind_speed', 'temperature'
original_df = pd.read_csv('../data/processed/climate_dengue_data.csv', parse_dates=['DATE'], index_col='DATE')
original_df.head()

,YEAR,MONTH,T2M,T2M_MAX,T2M_MIN,RH2M,PRECIPITATION,PS,GWETTOP,CASES
DATE,,,,,,,,,,
2000-01-01,2000,Jan,18.432581,25.855806,11.917742,62.302581,0.336129,101.303871,0.482903,0.0
2000-02-01,2000,Feb,20.534828,27.961379,13.949655,52.387586,0.767931,101.201724,0.418621,0.0
2000-03-01,2000,Mar,25.716129,33.292903,18.939677,55.770645,2.491290,100.850323,0.408387,0.0
2000-04-01,2000,Apr,29.433667,35.720000,24.302333,67.858667,5.166000,100.444667,0.414667,0.0
2000-05-01,2000,May,28.500000,32.106129,25.376452,81.627742,13.226774,100.290645,0.640000,0.0


In [11]:
if "Predicted" in original_df.columns:
    original_df.drop(['Predicted'],axis=1,inplace=True)

In [12]:
df = original_df.sort_values('DATE').copy()
df.shape

(288, 10)

In [13]:
# Check the first few rows of the dataframe
df.drop(columns=['YEAR','MONTH'], inplace=True)

# Combine 'TMPMAX' and 'TMPMIN' into 'TMPAVG'
df['TMPAVG'] = df[['T2M_MAX', 'T2M_MIN']].mean(axis=1)

# Drop the original columns if no longer needed
df = df.drop(columns=['T2M_MAX', 'T2M_MIN', "T2M"])

In [14]:
df.head()

,RH2M,PRECIPITATION,PS,GWETTOP,CASES,TMPAVG
DATE,,,,,,
2000-01-01,62.302581,0.336129,101.303871,0.482903,0.0,18.886774
2000-02-01,52.387586,0.767931,101.201724,0.418621,0.0,20.955517
2000-03-01,55.770645,2.491290,100.850323,0.408387,0.0,26.116290
2000-04-01,67.858667,5.166000,100.444667,0.414667,0.0,30.011167
2000-05-01,81.627742,13.226774,100.290645,0.640000,0.0,28.741290
